In [1]:
from data_utils import load_data, preprocess_data_faiss, preprocess_data_faiss_test

In [2]:
data = load_data("data")

In [3]:
embeddings = preprocess_data_faiss(data)
embeddings.shape

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["body"] = posts["body"].map(clean_html)


(11812, 2048)

In [4]:
posts, test_data = preprocess_data_faiss_test(data)
test_data

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["id"] = posts["id"].astype("int32")


,postid,relatedpostid,linktypeid
0,5620,3624,1
1,5129,4967,1
2,5800,174,1
3,3418,121,1
4,3317,1815,1
...,...,...,...
4267,174,970,1
4268,10224,10194,1
4269,5464,2711,1
4270,3706,3184,1


In [5]:
import faiss
import numpy as np

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
test_embeddings = embeddings[test_data["postid"]]

In [8]:
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

Recall@k (k = 5): 0.8625235404896422
Recall@k (k = 10): 0.9383911756793113
Recall@k (k = 15): 0.9607210115684692
Recall@k (k = 20): 0.9709443099273608


In [23]:
from sklearn.decomposition import PCA

In [30]:
pca = PCA(n_components=128)
reduced_embeddings = pca.fit_transform(embeddings)

In [67]:
random_embedding = reduced_embeddings[
    np.random.choice(reduced_embeddings.shape[0])
].reshape(1, -1)

In [68]:
index = faiss.IndexFlatL2(reduced_embeddings.shape[1])
index.add(reduced_embeddings)
test_embeddings = reduced_embeddings[test_data["postid"]]

print("IndexFlatL2:")
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

IndexFlatL2:
Recall@k (k = 5): 0.8810868980360506
Recall@k (k = 10): 0.9383911756793113
Recall@k (k = 15): 0.9704062415926823
Recall@k (k = 20): 0.9808985741189131


In [69]:
D, I = index.search(random_embedding, k=6)

print("IndexFlatL2:")
print(f"Distances: {D}")
print(f"Indices: {I}\n")

print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

IndexFlatL2:
Distances: [[0.         0.4257345  0.48082286 0.58143663 0.5844599  0.63861364]]
Indices: [[ 2912 10556  7816  9014 10547 11229]]

===ORIGINAL POST===
TITLE: Scratched Stainless Steel
CONTENT: I have a set of stainless steel pans that I absolutely love, and my 12" frying pan has received LOTS of use since I bought it. This is my first ever stainless steel pan and I'm pretty aware of all the ways to clean it (deglazing, barkeeps friend, etc). Recently, my wife purchased some scotch-bright heavy duty sponges with good intentions but I didn't realize exactly how "heavy duty" they were. I was washing my frying pan one night and used the scouring side of the sponge to get some stuck on grease and I noticed that I actually scratched my pan. The scratches don't look very deep, but I'm a little concerned.
Will these scratches damage the pan and reduce it's life? Should I stop using the scouring side of the sponges on my stainless steel?

===RECOMMENDATION 1 (distance = 0.425734490

In [70]:
index = faiss.IndexLSH(reduced_embeddings.shape[1], 64)
index.add(reduced_embeddings)
test_embeddings = reduced_embeddings[test_data["postid"]]

print("IndexLSH (n_bits=64):")
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

IndexLSH (n_bits=64):
Recall@k (k = 5): 0.8528383104654291
Recall@k (k = 10): 0.9464622006994888
Recall@k (k = 15): 0.9760559591068065
Recall@k (k = 20): 0.9905838041431262


In [71]:
D, I = index.search(random_embedding, k=6)

print("IndexLSH (n_bits=64):")
print(f"Distances: {D}")
print(f"Indices: {I}\n")

print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

IndexLSH (n_bits=64):
Distances: [[ 0. 11. 12. 13. 15. 15.]]
Indices: [[2912 7816 9847 1419 5067 9898]]

===ORIGINAL POST===
TITLE: Scratched Stainless Steel
CONTENT: I have a set of stainless steel pans that I absolutely love, and my 12" frying pan has received LOTS of use since I bought it. This is my first ever stainless steel pan and I'm pretty aware of all the ways to clean it (deglazing, barkeeps friend, etc). Recently, my wife purchased some scotch-bright heavy duty sponges with good intentions but I didn't realize exactly how "heavy duty" they were. I was washing my frying pan one night and used the scouring side of the sponge to get some stuck on grease and I noticed that I actually scratched my pan. The scratches don't look very deep, but I'm a little concerned.
Will these scratches damage the pan and reduce it's life? Should I stop using the scouring side of the sponges on my stainless steel?

===RECOMMENDATION 1 (distance = 11.0)===
TITLE: Stainless steel cookware scratches